In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import plotly.express as px


In [3]:
base_treino = pd.read_csv(r"D:\pipeline_dados\data_raw\base_orcamentos_treino.csv")
x_teste_df = pd.read_csv(r"D:\pipeline_dados\base_orcamentos_nov.csv")

In [15]:
x_teste_df

,dias_desejo,dias_em_casa,ds_tipo_obra,nm_prop_oportunidade,proprietario,promocional,tipo_loja,banheiro,toda_casa,cozinha,...,amb_outros,revestimento,officina,loucas_metais,arg_rejunte,categ_outros,vl_frete,vlr_orcamento,convertidos,nao_convertidos
0,4,23,Reforma,fabiano martins werutsky,zzzzzzzXzzzzzzzXzzzzzzzz,S,Tradicional A,S,N,N,...,S,1,0,0,1,0,63.33,823.62,343,438
1,7,23,Reforma,rafaela de sousa paonessa loureiro,zzzzzzzXzzXzzzzzXzzzzzzzzXzzzzzzzz,S,Tradicional A,N,S,N,...,S,2,0,0,0,0,83.33,1506.96,342,406
2,1,23,Reforma,rafaela de sousa paonessa loureiro,zzzzzzzXzzXzzzzzXzzzzzzzzXzzzzzzzz,S,Tradicional A,N,S,N,...,S,0,0,0,1,0,0.00,18.90,342,406
3,29,23,NaN,juliana westphalen gonçalves,zzzzzzzXzzzzzzzzzzXzzzzzzzzz,S,Tradicional A,N,S,N,...,S,0,1,0,3,0,106.67,20094.65,488,537
4,1,23,Reforma,rafaela de sousa paonessa loureiro,zzzzzzzXzzXzzzzzXzzzzzzzzXzzzzzzzz,S,Tradicional A,N,S,N,...,S,0,0,0,2,0,16.67,57.80,342,406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,1,0,Reforma,fabiano martins werutsky,zzzzzzzXzzzzzzzXzzzzzzzz,S,Tradicional A,N,N,N,...,S,1,0,0,2,0,56.67,199.90,343,438
236,7,0,Nova Obra,fabiano martins werutsky,zzzzzzzXzzzzzzzXzzzzzzzz,S,Tradicional A,S,N,N,...,N,6,0,6,0,0,0.00,4106.58,343,438
237,6,0,Reforma,matheus homrich,zzzzzzzXzzzzzzz,S,Tradicional A,N,N,N,...,S,1,0,0,4,0,215.00,3724.22,184,197
238,39,0,Reforma,juliana westphalen gonçalves,zzzzzzzXzzzzzzzzzzXzzzzzzzzz,S,Tradicional A,N,N,N,...,S,1,0,0,4,0,120.00,5788.80,488,537


In [4]:
px.histogram(x_teste_df, x = 'ds_tipo_obra', text_auto = True)

In [17]:
px.histogram(x_teste_df, x = 'convertidos', text_auto = True)

In [19]:
px.box(x_teste_df, x = 'vlr_orcamento', color = 'convertidos')

In [20]:
px.box(x_teste_df, x = 'vlr_orcamento', color = 'ds_tipo_obra')

In [5]:


y_teste = x_teste_df['convertido']
x_teste_df = x_teste_df.drop(['convertido', 'id_quote'], axis=1)

categoricos = ['ds_tipo_obra', 'nm_prop_oportunidade', 'proprietario', 'promocional', 'tipo_loja',
               'banheiro', 'toda_casa', 'cozinha', 'area_externa', 'itens_assentamento', 'sala',
               'piscina', 'area_servico', 'garagem', 'dormitorio', 'varanda', 'escada', 'amb_outros']

In [6]:
date_column = 'dt_criacao'

if date_column in base_treino.columns:
    print(f"Removing date column: {date_column}")
    base_treino = base_treino.drop(date_column, axis=1, errors='ignore')
    if date_column in categoricos:
        categoricos.remove(date_column)
else:
    print(f"Date column '{date_column}' not found in the training data.")

if date_column in x_teste_df.columns:
    x_teste_df = x_teste_df.drop(date_column, axis=1, errors='ignore')
    if date_column in categoricos:
        categoricos.remove(date_column)
else:
    print(f"Date column '{date_column}' not found in the test data.")

Removing date column: dt_criacao


In [7]:

X = base_treino.drop(['convertido'], axis=1, errors='ignore')
y = base_treino['convertido']

In [8]:

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

encoder = OneHotEncoder(drop='if_binary', handle_unknown='ignore')
X_encoded = pd.DataFrame(encoder.fit_transform(X[categoricos]).toarray(),
                          columns=encoder.get_feature_names_out(categoricos))

X_final = pd.concat([X.drop(categoricos, axis=1), X_encoded], axis=1)

X_test_encoded = pd.DataFrame(encoder.transform(x_teste_df[categoricos]).toarray(),
                               columns=encoder.get_feature_names_out(categoricos))
X_test_final = pd.concat([x_teste_df.drop('id_quote', axis=1, errors='ignore'), X_test_encoded], axis=1).drop(categoricos, axis=1).fillna(0)

X = X.select_dtypes(exclude=['datetime64[ns]']) 

d:\pipeline_dados\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:228: UserWarning:

Found unknown categories in columns [1, 2] during transform. These unknown categories will be encoded as all zeros



In [9]:
numeric_columns = X_final.select_dtypes(include=[np.number]).columns
var_threshold = 0.05
selector = VarianceThreshold(threshold=var_threshold)
X_high_variance = selector.fit_transform(X_final[numeric_columns])

selected_feature_indices = selector.get_support(indices=True)
X_resampled = X_high_variance
X_test_final = X_test_final.iloc[:, selected_feature_indices]

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_resampled, y)

In [10]:
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'min_samples_leaf': [1, 3, 5],
}


In [11]:

xgb_classifier = XGBClassifier()

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(xgb_classifier, param_grid, scoring='accuracy', cv=cv, n_jobs=-1)
grid_search.fit(X_resampled, y_resampled)

best_params = grid_search.best_params_

modelo_xgb = XGBClassifier(**best_params)

modelo_xgb.fit(X_resampled, y_resampled)

probabilidades_teste = modelo_xgb.predict_proba(X_test_final)[:, 1]


d:\pipeline_dados\venv\lib\site-packages\xgboost\core.py:160: UserWarning:

[20:21:22] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "min_samples_leaf" } are not used.


d:\pipeline_dados\venv\lib\site-packages\xgboost\core.py:160: UserWarning:

[20:21:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "min_samples_leaf" } are not used.




In [12]:
df_temporario_teste = pd.DataFrame({
    'nm_prop_oportunidade': x_teste_df.loc[x_teste_df.index, 'nm_prop_oportunidade'],
    'convertido': y_teste,
    'probabilidade_conversao': probabilidades_teste,
})


df_temporario_teste = df_temporario_teste.drop_duplicates(subset=['nm_prop_oportunidade', 'convertido'])
df_resultados_teste = df_temporario_teste.groupby('nm_prop_oportunidade', as_index=False).agg({
    'convertido': 'first',
    'probabilidade_conversao': 'mean'
})


with pd.option_context('display.max_rows', None):
    display(df_resultados_teste)

,nm_prop_oportunidade,convertido,probabilidade_conversao
0,adryson pinto freitas,S,0.477110
1,ediane rosa,S,0.920700
2,fabiano martins werutsky,S,0.518596
3,felipe pedroso alves,S,0.915357
4,juliana westphalen gonçalves,N,0.383864
5,matheus homrich,S,0.739461
6,michelle costa malta,N,0.746932
7,rafaela de sousa paonessa loureiro,S,0.723783
